#### Big query models : testing the infrastructure
- Only testing the mechanics!!!!

## CREATE OR REPLACE MODEL

In [8]:
%%bigquery
CREATE OR REPLACE MODEL `french_claims.v1`
OPTIONS (
    model_type='linear_reg',
    ls_init_learn_rate=.15,
    l1_reg=1,
    input_label_cols=['ClaimNb']
)
AS 
SELECT 
   VehPower,
   VehAge,
   DrivAge,
   BonusMalus,
   ClaimNb
FROM 
  `axa-ch-machine-learning-dev.french_claims.freMTPL2freq`
#where mod( Idpol,2)=0 #todo other splitting strat
;

""


## Model Info

In [9]:
%%bigquery
SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `french_claims.v1`)

,training_run,iteration,loss,eval_loss,duration_ms,learning_rate
0,0,2,0.057422,0.055640,18211,1.2
1,0,1,0.057703,0.055888,15772,0.6
2,0,0,0.058974,0.057122,11149,0.3


## Eval

In [12]:
%%bigquery
WITH eval_table AS (
SELECT
  *,
  ClaimNb AS label
FROM
  `axa-ch-machine-learning-dev.french_claims.freMTPL2freq`
  )

SELECT
  *
FROM
  ML.EVALUATE(MODEL `french_claims.v1`,    TABLE eval_table)
;

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,0.103422,0.057396,0.025242,0.054893,0.004486,0.004641


## Predict

In [15]:
%%bigquery
WITH to_predict AS (  SELECT    *   FROM      `axa-ch-machine-learning-dev.french_claims.freMTPL2freq` WHERE    DrivAge=20 )
SELECT
  truth.Idpol AS Idpol,
  predicted_ClaimNb
FROM 
  (SELECT    Idpol,predicted_ClaimNb FROM    ML.PREDICT(MODEL `french_claims.v1`,table to_predict) ) AS predict
  JOIN 
  (SELECT    Idpol  FROM    to_predict) AS truth
ON  predict.Idpol = truth.Idpol

,Idpol,predicted_ClaimNb
0,2147469.0,0.079633
1,4134782.0,0.067037
2,110410.0,0.083965
3,4150742.0,0.072735
4,4150743.0,0.072735
5,103299.0,0.081116
6,51027.0,0.101526
7,51028.0,0.101526
8,1114710.0,0.075701
9,1114711.0,0.073135


## own metric: Posson deviance

In [18]:
%%bigquery
WITH to_predict AS (  SELECT    *   FROM     `axa-ch-machine-learning-dev.french_claims.freMTPL2freq` )
select 2*avg(pois_dev) as mean_pois_dev from(
SELECT
  truth.Idpol AS Idpol,
  truth.Exposure AS exposure,
  truth.ClaimNb AS ClaimNb,
  predicted_ClaimNb ,
  (log(abs(power(truth.ClaimNb /predicted_ClaimNb*truth.Exposure,truth.ClaimNb))) )-
  (truth.ClaimNb-(predicted_ClaimNb*truth.Exposure)) as pois_dev
FROM 
  (SELECT    Idpol,predicted_ClaimNb FROM    ML.PREDICT(MODEL `french_claims.v1`,table to_predict) ) AS predict
  JOIN 
  (SELECT    Exposure,Idpol,ClaimNb  FROM    to_predict) AS truth
ON  predict.Idpol = truth.Idpol
)

,mean_pois_dev
0,0.184204
